In [47]:
import numpy as np
from pandas import DataFrame
import pandas as pd
import scipy as sp
from sklearn.naive_bayes import GaussianNB

Reading files: 

In [48]:
train_data = sp.genfromtxt("data/train_data.txt", delimiter="\t")

test_data = sp.genfromtxt("data/test_data.txt", delimiter="\t")

val_data =  sp.genfromtxt("data/val_data.txt", delimiter="\t")

Dimensions:

In [49]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(858L, 25L)
(286L, 25L)
(286L, 25L)


In [50]:
train_data = train_data[~np.isnan(train_data).any(axis=1)]
test_data = test_data[~np.isnan(test_data).any(axis=1)]
val_data = val_data[~np.isnan(val_data).any(axis=1)]
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(857L, 25L)
(286L, 25L)
(286L, 25L)


Brisanje proba: 

In [51]:
train_data[:1]

array([[  4.79000000e+02,   8.10000000e+01,   2.29000000e+01,
          4.08000000e+00,   7.65000000e+00,   5.34000000e-01,
          1.00000000e-02,   4.00000000e-02,   1.76000000e-01,
          2.72000000e+00,   3.45000000e+00,   7.88000000e-01,
          1.98000000e+00,   3.03000000e+00,   5.01000000e+00,
          6.80000000e-01,   4.30000000e-01,   4.10000000e-01,
          2.39000000e+00,   1.22000000e+00,   1.08900000e+01,
          8.00000000e+00,   3.40000000e+01,   1.12000000e+02,
          2.06000000e+02]])

In [52]:
np.delete(train_data[:1], 1)

array([  4.79000000e+02,   2.29000000e+01,   4.08000000e+00,
         7.65000000e+00,   5.34000000e-01,   1.00000000e-02,
         4.00000000e-02,   1.76000000e-01,   2.72000000e+00,
         3.45000000e+00,   7.88000000e-01,   1.98000000e+00,
         3.03000000e+00,   5.01000000e+00,   6.80000000e-01,
         4.30000000e-01,   4.10000000e-01,   2.39000000e+00,
         1.22000000e+00,   1.08900000e+01,   8.00000000e+00,
         3.40000000e+01,   1.12000000e+02,   2.06000000e+02])

# Features

In [53]:
#train_features = np.delete(train_data, 22, 1)
#test_features = np.delete(test_data, 22, 1)
#val_features = np.delete(val_data,22,1)
                          
#train_features = np.delete(train_features, 21, 1)
#test_features = np.delete(test_features, 21, 1)
#val_features = np.delete(val_features,21,1)
                          
#print(train_features.shape)
#print(test_features.shape)
#print(val_features.shape)

columns = {"GP":0, "GS":1, "MIN":2, "FGM":3,"FGA":4,"FG%":5,"3PM":6,"3PA":7,"3P%":8,"FTM":9,"FTA":10,"FT%":11,"OFF":12,"DEF":13,
              "TRB":14,"AST":15,"STL":16,"BLK":17,"PF":18,"TOV":19,"PTS":20,"YR":21,"POS":22,"W":23,"H":24} 


In [99]:
fl = [columns["GP"],columns["GS"],columns["MIN"],columns["FG%"],
      columns["3P%"],columns["FT%"],columns["PTS"],columns["YR"],columns["POS"],columns['3PM'],columns['FTM'],columns['FGM']]

In [98]:
train_features = np.delete(train_data,fl,1)
test_features = np.delete(test_data,fl,1)
val_features = np.delete(val_data,fl,1)


In [56]:
print "Training features dimensions: "  
print train_features.shape
print "Test features dimensions: " 
print test_features.shape
print "Validation features dimensions: "
print val_features.shape

Training features dimensions: 
(857L, 13L)
Test features dimensions: 
(286L, 13L)
Validation features dimensions: 
(286L, 13L)


# Labels

In [57]:
train_labels = train_data[:,22]
test_labels = test_data[:,22]
val_labels = val_data[:,22]
print(train_labels.shape)
print(test_labels.shape)
print(val_labels.shape)

(857L,)
(286L,)
(286L,)


# Naive Bayes

In [58]:
clf = GaussianNB()

In [59]:
clf.fit(train_features, train_labels)

GaussianNB()

In [60]:
pred = clf.predict(test_features)
pred

array([ 12.,   1.,   5.,  45.,   3.,  34.,   4.,   5.,  12.,  34.,  12.,
        34.,   4.,   4.,  12.,  12.,  12.,   5.,  12.,   5.,  34.,   3.,
         5.,  34.,  12.,  12.,  34.,   1.,  12.,   1.,   4.,  34.,  12.,
        34.,  12.,  12.,   5.,  34.,  12.,  12.,   5.,   5.,  34.,  23.,
         3.,  34.,  12.,  12.,   5.,  12.,   5.,  45.,  34.,  12.,   5.,
         5.,  34.,  12.,   5.,  12.,   5.,   5.,   5.,  12.,  34.,  34.,
        34.,  12.,  12.,  34.,  12.,   4.,   2.,   5.,  12.,  12.,  12.,
        34.,  45.,   5.,  12.,  23.,  12.,   2.,   1.,   3.,  34.,  23.,
         5.,  34.,  12.,   1.,  12.,   3.,  34.,  12.,  23.,  23.,  34.,
         5.,  12.,  23.,  45.,   2.,   3.,   5.,  34.,   4.,  34.,  23.,
         5.,   5.,  12.,  23.,   1.,  12.,   5.,  34.,   3.,   5.,  23.,
         5.,  34.,   3.,   2.,   4.,   4.,  34.,   1.,   5.,  34.,   5.,
         5.,   1.,  45.,   5.,   1.,   4.,  12.,  45.,   4.,   1.,  34.,
        23.,   4.,   1.,  12.,   2.,  23.,   5.,   

In [61]:
test_labels

array([  2.,   1.,   5.,   5.,   2.,   3.,   4.,  45.,  12.,   4.,  12.,
        34.,   4.,  45.,  12.,   2.,  12.,  45.,   2.,  45.,  34.,   3.,
         5.,  34.,   3.,   2.,  34.,   1.,  12.,   1.,   5.,  34.,  12.,
         4.,  12.,   2.,   4.,  34.,   2.,  12.,   4.,  34.,  34.,   3.,
         3.,   4.,  12.,  12.,   4.,   2.,   5.,  45.,  34.,  12.,   4.,
         5.,  34.,  12.,  45.,  12.,   5.,   5.,   5.,  12.,  34.,  34.,
        34.,  12.,  12.,   4.,   2.,   4.,   2.,   5.,  12.,   2.,   2.,
        45.,   5.,   5.,  12.,   2.,   1.,   2.,   1.,   2.,  34.,  34.,
         5.,  34.,  12.,   2.,   2.,   2.,  34.,  34.,   3.,  34.,   4.,
         4.,  12.,  34.,  45.,   2.,  23.,   4.,  34.,   4.,  34.,  34.,
        34.,   4.,  12.,  12.,   1.,  12.,   5.,  34.,   3.,   5.,   3.,
        45.,  34.,   3.,   2.,   4.,   4.,  34.,   1.,   5.,   4.,   5.,
         4.,   1.,   5.,   5.,   1.,   4.,   2.,   4.,   4.,   2.,  34.,
         3.,   4.,  12.,  12.,   2.,   3.,   5.,  3

# Accuracy

In [62]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(pred, test_labels)
accuracy

0.59790209790209792

# Custom Accuracy function

In [63]:
def custom_acc(pred_labels, test_labels):
    hit = 0
    num = 0
    j = 0
    for i in pred_labels:
        a = i
        a = str(int(a))
        b = test_labels[j]
        b = str(int(b))
        if a == b:
            hit = hit+1
        elif len(a) == 1 and len(b) == 2:
            if a == b[0] or a == b[1]:
                hit = hit+1
        elif len(a) == 2 and len(b) == 1:
            if a[0] == b or a[1] == b:
                hit = hit+1
        num = num+1
        j = j+1
    print num
    print hit
    return float(hit)/num


# Custom Accuracy NB

In [64]:
custom_acc(pred, test_labels)

286
241


0.8426573426573427

# Confusion matrix NB

In [88]:
from sklearn.metrics import confusion_matrix 

In [90]:
confm_nb = confusion_matrix(test_labels, pred)
confm_nb
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[12,  0,  0,  0,  0,  2,  0,  0,  0],
       [ 2,  7,  3,  0,  0, 22,  2,  0,  0],
       [ 0,  0,  9,  0,  0,  1,  7,  3,  0],
       [ 0,  0,  0, 15, 13,  0,  0,  9,  2],
       [ 0,  0,  0,  1, 38,  0,  0,  0,  8],
       [ 2,  1,  0,  0,  0, 54,  2,  0,  0],
       [ 0,  0,  2,  0,  0,  1,  2,  0,  0],
       [ 0,  0,  1,  0,  8,  6,  7, 32,  0],
       [ 0,  0,  0,  1,  8,  0,  0,  1,  2]])

# Validation     

In [65]:
pred2 = clf.predict(val_features)
pred2

array([ 12.,   1.,   5.,  45.,   3.,  34.,   4.,   5.,  12.,  34.,  12.,
        34.,   4.,   4.,  12.,  12.,  12.,   5.,  12.,   5.,  34.,   3.,
         5.,  34.,  12.,  12.,  34.,   1.,  12.,   1.,   4.,  34.,  12.,
        34.,  12.,  12.,   5.,  34.,  12.,  12.,   5.,   5.,  34.,  23.,
         3.,  34.,  12.,  12.,   5.,  12.,   5.,  45.,  34.,  12.,   5.,
         5.,  34.,  12.,   5.,  12.,   5.,   5.,   5.,  12.,  34.,  34.,
        34.,  12.,  12.,  34.,  12.,   4.,   2.,   5.,  12.,  12.,  12.,
        34.,  45.,   5.,  12.,  23.,  12.,   2.,   1.,   3.,  34.,  23.,
         5.,  34.,  12.,   1.,  12.,   3.,  34.,  12.,  23.,  23.,  34.,
         5.,  12.,  23.,  45.,   2.,   3.,   5.,  34.,   4.,  34.,  23.,
         5.,   5.,  12.,  23.,   1.,  12.,   5.,  34.,   3.,   5.,  23.,
         5.,  34.,   3.,   2.,   4.,   4.,  34.,   1.,   5.,  34.,   5.,
         5.,   1.,  45.,   5.,   1.,   4.,  12.,  45.,   4.,   1.,  34.,
        23.,   4.,   1.,  12.,   2.,  23.,   5.,   

In [66]:
accuracy2 = accuracy_score(pred2, val_labels)
accuracy2

0.59790209790209792

In [67]:
custom_acc(pred2, val_labels)

286
241


0.8426573426573427

# Feature selection with VarianceTheshold 

In [68]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))


In [69]:
train_features.shape

(857L, 13L)

In [70]:
sel.fit_transform(train_features)[:1]

array([[  7.65000000e+00,   4.00000000e-02,   3.45000000e+00,
          1.98000000e+00,   3.03000000e+00,   5.01000000e+00,
          6.80000000e-01,   4.10000000e-01,   2.39000000e+00,
          1.22000000e+00,   1.12000000e+02,   2.06000000e+02]])

In [71]:
train_features1 = sel.fit_transform(train_features)
val_features1 = np.delete(val_features, 11, 1)
train_features1.shape
train_labels.shape

(857L,)

In [72]:
clf.fit(val_features1, val_labels)

GaussianNB()

# Accuracy with validation set NB

In [73]:
pred3 = clf.predict(val_features1)
accuracy3 = accuracy_score(pred3, val_labels)
accuracy3

0.61888111888111885

# Custom accuracy with validation set NB

In [91]:
custom_acc(pred3, val_labels)

286
244


0.8531468531468531

# Confusion matrix with validation set 

In [94]:
confm_nb_val = confusion_matrix(val_labels, pred3)
confm_nb_val
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[11,  0,  0,  0,  0,  3,  0,  0,  0],
       [ 4,  9,  4,  0,  0, 16,  3,  0,  0],
       [ 0,  1, 13,  0,  0,  2,  1,  3,  0],
       [ 0,  0,  0, 16,  3,  0,  0, 15,  5],
       [ 0,  0,  0,  2, 32,  0,  0,  6,  7],
       [ 3,  4,  1,  0,  0, 51,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  2,  3,  0,  0],
       [ 0,  0,  2,  2,  0,  6,  1, 39,  4],
       [ 0,  0,  0,  2,  1,  0,  0,  6,  3]])

# SVM

In [74]:
from sklearn.svm import SVC

In [75]:
clfSVM= SVC()

In [76]:
clfSVM.fit(train_features, train_labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [77]:
print train_features.shape
print train_labels.shape

(857L, 13L)
(857L,)


In [78]:
test_features.shape

(286L, 13L)

In [79]:
predSVM = clfSVM.predict(test_features)

# Accuracy SVM

In [80]:
accuracySVM = accuracy_score(predSVM, test_labels)
accuracySVM

0.50349650349650354

# Custom Accuracy SVM

In [81]:
custom_acc(predSVM, test_labels)

286
247


0.8636363636363636

# Confusion matrix SVM

In [95]:
confm_svm = confusion_matrix(test_labels, predSVM)
confm_svm
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[ 2,  0,  0,  0,  0, 12,  0,  0,  0],
       [ 0,  3,  2,  0,  0, 30,  1,  0,  0],
       [ 0,  2,  2,  0,  0,  4,  0, 12,  0],
       [ 0,  0,  0,  9,  6,  2,  0, 18,  4],
       [ 0,  0,  0,  4, 40,  1,  0,  1,  1],
       [ 6,  0,  3,  0,  0, 50,  0,  0,  0],
       [ 0,  3,  0,  0,  0,  1,  1,  0,  0],
       [ 0,  0,  6,  3,  4,  2,  1, 36,  2],
       [ 0,  0,  0,  2,  5,  0,  0,  4,  1]])

# Logistic Regression

In [82]:
from sklearn import linear_model

In [83]:
logreg = linear_model.LogisticRegression(C=1e5)

In [84]:
logreg.fit(train_features, train_labels)

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [85]:
predLR = logreg.predict(test_features)

# Logistic Regression Accuracy

In [86]:
accLR = accuracy_score(predLR, test_labels)
accLR

0.56993006993006989

# Custom Accuracy SVM

In [87]:
custom_acc(predLR, test_labels)

286
240


0.8391608391608392

# Confusion matrix LR

In [96]:
confm_lr = confusion_matrix(test_labels, predLR)
confm_lr
#  labels 1 - 2 - 3- 4 - 5 - 12- 23- 34 - 45

array([[ 7,  0,  0,  0,  0,  7,  0,  0,  0],
       [ 0,  5,  1,  0,  0, 25,  1,  4,  0],
       [ 1,  3,  1,  0,  0,  3,  0, 12,  0],
       [ 0,  0,  0, 12,  9,  0,  0, 17,  1],
       [ 0,  0,  0,  1, 43,  0,  0,  2,  1],
       [ 2,  1,  0,  0,  0, 52,  0,  4,  0],
       [ 0,  1,  0,  0,  0,  2,  0,  2,  0],
       [ 0,  0,  0,  3,  2,  6,  1, 42,  0],
       [ 0,  0,  0,  1,  5,  0,  0,  5,  1]])